In [ ]:
import socket, threading, operations as op, stats as st

sf, sn, sw = 0, 0, 4  # Sliding window variables
flag, flag2 = False, False  # Flags for control flow
data_queue = [None] * sw

# Initialize socket
sender = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sender.bind(st.ADDR)

def send_frames(conn):
    global sn, sf, sw, flag, data_queue
    while not flag:
        if sn - sf < sw:
            try:
                data = text.pop(0)  # Get the next data to send
            except IndexError:
                print("[FINISHED] All data sent.")
                flag = True
                return

            frame = op.noisychannel(op.makeFrame(sn, data))
            print(f"[SENDING] Frame {sn}: {frame}")
            conn.send(frame.encode())
            data_queue[sn % sw] = data
            sn += 1

def recv_ack(conn):
    global sf, sn, sw, data_queue, flag2
    while not flag2:
        try:
            conn.settimeout(0.5)
            recv_frame = conn.recv(20).decode()
            ackNo, _, data, _ = op.receiveFrame(recv_frame)
        except socket.timeout:
            resend_frames(conn)
            continue

        if sf <= ackNo < sn and data == '11110000':  # Valid ACK
            while sf <= ackNo:
                print(f"[ACK RECEIVED] ACK {ackNo}")
                data_queue[sf % sw] = None
                sf += 1

        if ackNo == length - 1:  # All ACKs received
            flag2 = True

def resend_frames(conn):
    for i in range(sf, sn):
        data = data_queue[i % sw]
        frame = op.makeFrame(i, data)
        print(f"[RESENDING] Frame {i}: {frame}")
        conn.send(frame.encode())

def start_server():
    global text, length
    sender.listen()
    print(f"[LISTENING] Server listening on {st.HOST_IP}")
    
    conn, addr = sender.accept()
    print(f"[CONNECTED] Connected to {addr}")

    with open("data.txt", "r") as f:
        text = [line.strip() for line in f.readlines()]
    
    length = len(text)
    
    sender_thread = threading.Thread(target=send_frames, args=(conn,))
    receiver_thread = threading.Thread(target=recv_ack, args=(conn,))
    
    sender_thread.start()
    receiver_thread.start()
    sender_thread.join()
    receiver_thread.join()

    print("[CLOSING] Closing connection...")
    conn.close()

start_server()
